In [ ]:
pip install pyserial sympy numpy

In [ ]:
# Braccio机器人逆向运动学实验 - 基于参考代码精确实现
# 严格按照参考程序的DH参数和角度补偿

import serial
import time
from sympy import *
import numpy as np

print("=== Braccio机器人逆向运动学实验 ===")
print("基于参考代码的精确实现")

# ============================================================================
# 第一部分：从参考代码复制的DH变换矩阵和变换函数
# ============================================================================

# 定义基本DH变换矩阵（与参考代码完全一致）
var('d,a,alpha,delta,Rz,Rx,Tz,Tx')

Rx=Matrix([[1,0,0,0],
          [0,cos(alpha),-sin(alpha),0],
          [0,sin(alpha),cos(alpha),0],
          [0,0,0,1]])
Rz=Matrix([[cos(delta),-sin(delta),0,0],
          [sin(delta),cos(delta),0,0],
          [0,0,1,0],
          [0,0,0,1]])
Tz=Matrix([[1,0,0,0],
          [0,1,0,0],
          [0,0,1,d],
          [0,0,0,1]])
Tx=Matrix([[1,0,0,a],
          [0,1,0,0],
          [0,0,1,0],
          [0,0,0,1]])
TM=Tz.multiply(Rz).multiply(Tx).multiply(Rx)

# 度数转弧度函数（与参考代码一致）
def d2r(deg):
    return deg/180*pi

# 各关节变换函数（完全复制参考代码）
def Tv1n0(delta1):
    var('delta1,d1,alpha1,a1')
    #DH-Parameter
    d1=70
    #convert angle from degree to rad and compensate the offset from the home position
    delta1=d2r(delta1) + pi - 0
    a1=0    
    alpha1=pi/2
    return TM.subs({delta:delta1,d:d1,alpha:alpha1,a:a1})

def Tv2n1(delta2):
    var('delta2,d2,alpha2,a2')
    #DH-Parameter
    d2=0
    #convert angle from degree to rad and compensate the offset from the home position
    delta2=d2r(delta2)
    a2=120
    alpha2=0
    return TM.subs({delta:delta2,d:d2,alpha:alpha2,a:a2})

def Tv3n2(delta3):
    var('delta3,d3,alpha3,a3')
    #DH-Parameter
    d3=0
    #convert angle from degree to rad and compensate the offset from the home position
    delta3=d2r(delta3) - pi/2
    a3=120    
    alpha3=0
    return TM.subs({delta:delta3,d:d3,alpha:alpha3,a:a3})

def Tv4n3(delta4):
    var('delta4,d4,alpha4,a4')
    #DH-Parameter
    d4=0
    #convert angle from degree to rad and compensate the offset from the home position
    delta4=d2r(delta4)-pi/2
    a4=80    
    alpha4=-pi/2
    return TM.subs({delta:delta4,d:d4,alpha:alpha4,a:a4})

def Tv5n4(delta5):
    var('delta5,d5,alpha5,a5')
    #DH-Parameter
    d5=150
    #convert angle from degree to rad and compensate the offset from the home position
    delta5=d2r(delta5)-pi/2
    a5=0    
    alpha5=0
    return TM.subs({delta:delta5,d:d5,alpha:alpha5,a:a5})

# 正向运动学函数（与参考代码一致）
def BraccioForward(base, shoulder, elbow, wrist, twist):
    return Tv1n0(base)*Tv2n1(shoulder)*Tv3n2(elbow)*Tv4n3(wrist)*Tv5n4(twist)

print("✓ DH变换函数已定义（基于参考代码）")

# ============================================================================
# 第二部分：从参考代码提取的全局几何参数
# ============================================================================

# 从参考代码的变换函数中提取的几何参数
d1 = 70   # 基座高度
a2 = 120  # 第二关节臂长
a3 = 120  # 第三关节臂长  
a4 = 80   # 第四关节臂长
d5 = 150  # 末端执行器长度

print(f"✓ 几何参数: d1={d1}, a2={a2}, a3={a3}, a4={a4}, d5={d5}")

# ============================================================================
# 第三部分：串行通信（与参考代码一致）
# ============================================================================

def establish_serial_connection():
    """建立串行连接 - 与参考代码一致的设置"""
    try:
        port = input("请输入COM端口号（例如：COM7，参考代码默认COM7）: ").strip()
        if not port:
            port = 'COM7'  # 参考代码中的默认端口
        
        # 与参考代码完全一致的串行设置
        s = serial.Serial(port, 115200, timeout=5)
        time.sleep(3)  # 参考代码中的等待时间
        print(f"✓ 成功连接到 {port}")
        return s
    except Exception as e:
        print(f"✗ 连接失败: {e}")
        return None

def move_to_home_position(serial_conn):
    """移动到初始位置 - 与参考代码命令一致"""
    if serial_conn:
        # 参考代码中的初始位置命令
        command = b'P90,90,0,90,0,100,30\n'
        serial_conn.write(command)
        response = serial_conn.readline().decode().strip()
        print(f"机器人响应: {response}")
        return response == "OK"
    return False

# ============================================================================
# 第四部分：逆向运动学实现
# ============================================================================

def BraccioInverse(w):
    """
    Braccio机器人逆向运动学函数
    参数: w = [x, y, z] - 目标位置向量
    返回: q = [None, q1, q2, q3, q4, q5] - 关节角度向量（度数）
    注意: 索引从1开始，符合实验要求
    """
    
    # 创建关节角度向量，索引从1开始（实验要求）
    q = [None] * 6  # q[0]不使用，q[1]到q[5]存储关节角度
    
    x, y, z = float(w[0]), float(w[1]), float(w[2])
    
    print(f"\n--- 逆向运动学计算 ---")
    print(f"目标位置: x={x}, y={y}, z={z}")
    
    try:
        # 步骤1: 计算 q[1] (基座旋转角) - 度数
        if x == 0 and y == 0:
            q[1] = 0  # 如果在原点正上方
        else:
            q[1] = float(atan2(y, x) * 180 / pi)
        print(f"q[1] (基座) = {q[1]:.2f}°")
        
        # 步骤2: 计算到目标点的平面距离
        r = sqrt(x**2 + y**2)
        print(f"平面距离 r = {float(r):.2f}")
        
        # 步骤3: 计算有效的z坐标（减去基座高度）
        z_eff = z - d1
        print(f"有效z坐标 (z - d1) = {z_eff:.2f}")
        
        # 步骤4: 计算到腕关节的距离（减去末端长度的投影）
        # 假设末端执行器垂直向下
        r_wrist = r
        z_wrist = z_eff + d5  # 腕关节比末端高d5
        
        print(f"腕关节位置: r_wrist={r_wrist:.2f}, z_wrist={z_wrist:.2f}")
        
        # 步骤5: 到腕关节的直线距离
        dist_to_wrist = sqrt(r_wrist**2 + z_wrist**2)
        print(f"到腕关节的距离 = {float(dist_to_wrist):.2f}")
        
        # 步骤6: 使用余弦定理计算 q[3] (肘关节角)
        cos_q3 = (a2**2 + a3**2 - dist_to_wrist**2) / (2 * a2 * a3)
        
        if abs(cos_q3) > 1:
            print(f"✗ 余弦值超出范围: {float(cos_q3):.3f}")
            return None
            
        q[3] = float(acos(cos_q3) * 180 / pi)
        print(f"q[3] (肘关节) = {q[3]:.2f}°")
        
        # 步骤7: 计算 q[2] (肩关节角)
        beta = atan2(z_wrist, r_wrist)
        alpha = atan2(a3 * sin(q[3] * pi / 180), a2 + a3 * cos(q[3] * pi / 180))
        q[2] = float((beta - alpha) * 180 / pi)
        print(f"q[2] (肩关节) = {q[2]:.2f}°")
        
        # 步骤8: 计算 q[4] (腕关节角) - 保持末端垂直向下
        q[4] = 90 - (q[2] + q[3])  # 保持末端垂直
        print(f"q[4] (腕关节) = {q[4]:.2f}°")
        
        # 步骤9: q[5] (扭转角) - 通常设为0
        q[5] = 0
        print(f"q[5] (扭转) = {q[5]:.2f}°")
        
        return q
        
    except Exception as e:
        print(f"✗ 逆向运动学计算错误: {e}")
        return None

def genBraccioString(q):
    """
    生成发送给机器人控制器的命令字符串
    参数: q - 关节角度向量（度数）
    返回: 命令字符串或None（如果位置不可达）
    
    根据参考代码中的角度补偿进行调整
    """
    
    if q is None:
        return None
    
    print(f"\n--- 生成机器人命令 ---")
    
    # 应用参考代码中的角度补偿
    # 注意：参考代码中的补偿是在正向运动学中应用的，逆向时需要反向补偿
    
    # base: 参考代码中 delta1 = d2r(delta1) + pi，所以逆向时需要减去180°
    base_cmd = q[1] + 180  # 加180度补偿
    if base_cmd > 360:
        base_cmd -= 360
    if base_cmd < 0:
        base_cmd += 360
    
    # shoulder: 参考代码中 delta2 = d2r(delta2)，无补偿
    shoulder_cmd = q[2]
    
    # elbow: 参考代码中 delta3 = d2r(delta3) - pi/2，所以逆向时需要加90°
    elbow_cmd = q[3] + 90
    
    # wrist: 参考代码中 delta4 = d2r(delta4) - pi/2，所以逆向时需要加90°
    wrist_cmd = q[4] + 90
    
    # twist: 参考代码中 delta5 = d2r(delta5) - pi/2，所以逆向时需要加90°
    twist_cmd = q[5] + 90
    
    print(f"补偿后角度: base={base_cmd:.1f}°, shoulder={shoulder_cmd:.1f}°, elbow={elbow_cmd:.1f}°, wrist={wrist_cmd:.1f}°, twist={twist_cmd:.1f}°")
    
    # 检查角度范围（0-180度）
    angles = [base_cmd, shoulder_cmd, elbow_cmd, wrist_cmd, twist_cmd]
    angle_names = ['base', 'shoulder', 'elbow', 'wrist', 'twist']
    
    for i, (angle, name) in enumerate(zip(angles, angle_names)):
        if angle < 0 or angle > 180:
            print(f"✗ {name}角度 = {angle:.1f}° 超出伺服电机范围 [0°, 180°]")
            return None
    
    # 生成命令字符串（与参考代码格式一致）
    command = f"P{int(base_cmd)},{int(shoulder_cmd)},{int(elbow_cmd)},{int(wrist_cmd)},{int(twist_cmd)},130,50"
    
    print(f"✓ 生成的命令: {command}")
    return command

# ============================================================================
# 第五部分：测试和验证
# ============================================================================

def verify_with_forward_kinematics(q):
    """使用正向运动学验证逆向运动学结果"""
    if q is None:
        return None
    
    print(f"\n--- 正向运动学验证 ---")
    
    # 使用参考代码的正向运动学函数
    # 注意：需要应用相同的角度补偿
    base_fwd = q[1] + 180
    shoulder_fwd = q[2]
    elbow_fwd = q[3] + 90
    wrist_fwd = q[4] + 90
    twist_fwd = q[5] + 90
    
    # 确保角度在有效范围内
    if any(angle < 0 or angle > 180 for angle in [base_fwd, shoulder_fwd, elbow_fwd, wrist_fwd, twist_fwd]):
        print("角度超出范围，无法进行正向验证")
        return None
    
    try:
        # 使用参考代码的正向运动学
        P5 = Matrix([0, 0, 0, 1])  # 末端执行器坐标系原点
        P0 = BraccioForward(base_fwd, shoulder_fwd, elbow_fwd, wrist_fwd, twist_fwd) * P5
        result = N(P0)
        
        print(f"正向运动学结果:")
        print(f"x = {float(result[0]):.2f}")
        print(f"y = {float(result[1]):.2f}")
        print(f"z = {float(result[2]):.2f}")
        
        return [float(result[0]), float(result[1]), float(result[2])]
        
    except Exception as e:
        print(f"正向运动学验证失败: {e}")
        return None

def test_reference_point():
    """测试参考代码中的已知点"""
    print("\n=== 测试参考代码中的已知点 ===")
    
    # 参考代码中的测试：BraccioForward(0,90,180,90,0) -> [200, 0, 40]
    print("参考代码测试点:")
    print("输入角度: base=0°, shoulder=90°, elbow=180°, wrist=90°, twist=0°")
    print("预期输出: [200, 0, 40]")
    
    # 使用我们的逆向运动学计算 [200, 0, 40]
    target = [200, 0, 40]
    q = BraccioInverse(target)
    
    if q is not None:
        # 验证结果
        verification = verify_with_forward_kinematics(q)
        
        if verification is not None:
            error_x = abs(verification[0] - target[0])
            error_y = abs(verification[1] - target[1])
            error_z = abs(verification[2] - target[2])
            
            print(f"\n误差分析:")
            print(f"X误差: {error_x:.2f}mm")
            print(f"Y误差: {error_y:.2f}mm")  
            print(f"Z误差: {error_z:.2f}mm")
            print(f"总误差: {sqrt(error_x**2 + error_y**2 + error_z**2):.2f}mm")
            
            if max(error_x, error_y, error_z) < 5:  # 5mm容差
                print("✓ 逆向运动学验证成功！")
            else:
                print("⚠ 误差较大，需要调整算法")
    
    return q

def interactive_test(serial_conn):
    """交互式测试"""
    print("\n=== 交互式测试模式 ===")
    print("输入目标位置坐标，程序将计算逆向运动学并可选择控制机器人")
    print("输入格式: x,y,z （例如: 200,0,40）")
    print("输入 'test' 运行参考点测试")
    print("输入 'quit' 退出")
    
    while True:
        try:
            user_input = input("\n请输入: ").strip()
            
            if user_input.lower() == 'quit':
                break
            elif user_input.lower() == 'test':
                test_reference_point()
                continue
            
            # 解析输入坐标
            coords = [float(x.strip()) for x in user_input.split(',')]
            
            if len(coords) != 3:
                print("请输入三个坐标值: x,y,z")
                continue
            
            print(f"\n目标位置: [{coords[0]}, {coords[1]}, {coords[2]}]")
            
            # 计算逆向运动学
            q = BraccioInverse(coords)
            
            if q is not None:
                # 验证结果
                verification = verify_with_forward_kinematics(q)
                
                # 生成命令
                command = genBraccioString(q)
                
                if command is not None:
                    print(f"\n✓ 计算成功！")
                    
                    # 发送命令到机器人
                    if serial_conn:
                        send_cmd = input("是否发送命令到机器人？(y/n): ").strip().lower()
                        if send_cmd == 'y':
                            try:
                                serial_conn.write(f"{command}\n".encode('ascii'))
                                response = serial_conn.readline().decode().strip()
                                print(f"机器人响应: {response}")
                                
                                if response == "OK":
                                    print("✓ 命令发送成功")
                                    print("请用卡尺测量实际位置并与目标位置对比")
                                else:
                                    print("⚠ 机器人响应异常")
                            except Exception as e:
                                print(f"发送命令失败: {e}")
                        else:
                            print("命令未发送")
                    else:
                        print("串行连接未建立，无法发送命令")
                else:
                    print("✗ 角度超出机器人工作范围")
            else:
                print("✗ 目标位置不可达")
                
        except ValueError:
            print("输入格式错误，请使用 x,y,z 格式")
        except Exception as e:
            print(f"错误: {e}")

# ============================================================================
# 第六部分：主程序
# ============================================================================

def main():
    """主程序"""
    
    print("\n开始Braccio机器人逆向运动学实验")
    print("基于参考代码的精确实现")
    
    # 首先测试参考点
    print("\n--- 第一步：验证参考点 ---")
    test_reference_point()
    
    # 询问是否连接实际机器人
    connect_robot = input("\n是否连接实际机器人进行测试？(y/n): ").strip().lower()
    
    serial_conn = None
    if connect_robot == 'y':
        serial_conn = establish_serial_connection()
        
        if serial_conn:
            print("\n--- 移动到初始位置 ---")
            if move_to_home_position(serial_conn):
                print("✓ 机器人已移动到初始位置")
                time.sleep(2)  # 等待运动完成
            else:
                print("✗ 移动到初始位置失败")
    
    # 交互式测试
    interactive_test(serial_conn)
    
    # 关闭串行连接
    if serial_conn:
        serial_conn.close()
        print("\n串行连接已关闭")
    
    print("实验结束")

# 运行主程序
if __name__ == "__main__":
    main()